In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.io as pio

# Set the default renderer to open the plot in a browser
pio.renderers.default = "browser"

# Load the ESG data (replace with your file path)
file_path = r"C:/Users/rajas/sp500_esg_data.xlsx"
esg_data = pd.read_excel(file_path, sheet_name='sp500_esg_data')

# Select governance-related columns: 'governanceScore', 'CEO Tenure', '% Indep Dir on Board' (if available)
# Assuming we only have 'governanceScore' from this dataset
governance_data = esg_data[['governanceScore']].dropna()

# Standardize the data for PCA
scaler = StandardScaler()
governance_data_scaled = scaler.fit_transform(governance_data)

# Apply PCA
pca = PCA(n_components=1)  # We only have one governance variable here
pca_result = pca.fit_transform(governance_data_scaled)

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(pca_result, columns=['Principal Component 1'])

# Concatenate PCA results with the original company symbols for easier interpretation
pca_merged_data = pd.concat([esg_data[['Symbol']], pca_df], axis=1)

# Display the results
print(pca_merged_data.head())

# Strategy: Long stocks with high PC1 (strong governance), Short stocks with low PC1 (weak governance)
long_positions = pca_merged_data[pca_merged_data['Principal Component 1'] > 0]
short_positions = pca_merged_data[pca_merged_data['Principal Component 1'] < 0]

print("\nLong Positions (High Governance Scores):")
print(long_positions)

print("\nShort Positions (Low Governance Scores):")
print(short_positions)

# Create an interactive plot
fig = go.Figure()

# Add scatter plot for long positions
fig.add_trace(go.Scatter(
    x=long_positions.index,
    y=long_positions['Principal Component 1'],
    mode='markers',
    name='Long Positions',
    marker=dict(color='green', size=8),
    text=long_positions['Symbol'],
    hovertemplate='<b>%{text}</b><br>PCA Score: %{y:.2f}<extra></extra>'
))

# Add scatter plot for short positions
fig.add_trace(go.Scatter(
    x=short_positions.index,
    y=short_positions['Principal Component 1'],
    mode='markers',
    name='Short Positions',
    marker=dict(color='red', size=8),
    text=short_positions['Symbol'],
    hovertemplate='<b>%{text}</b><br>PCA Score: %{y:.2f}<extra></extra>'
))

# Update layout
fig.update_layout(
    title='Governance PCA Strategy: Long vs Short Positions',
    xaxis_title='Company Index',
    yaxis_title='Principal Component 1 (Governance Score)',
    hovermode='closest'
)

# Show the plot
fig.show()


  Symbol  Principal Component 1
0      A              -0.277875
1    AAL              -0.897421
2   AAPL               1.234002
3   ABBV               1.691726
4    ABT               0.753160

Long Positions (High Governance Scores):
    Symbol  Principal Component 1
2     AAPL               1.234002
3     ABBV               1.691726
4      ABT               0.753160
5     ACGL               1.936770
8      ADI               0.128991
..     ...                    ...
417    WTW               1.876665
419   WYNN               1.793443
421    XOM               0.822512
424    ZBH               0.549727
425    ZTS               0.933476

[166 rows x 2 columns]

Short Positions (Low Governance Scores):
    Symbol  Principal Component 1
0        A              -0.277875
1      AAL              -0.897421
6      ACN              -1.059243
7     ADBE              -0.717106
9      ADM              -0.781834
..     ...                    ...
414    WMB              -1.729647
418     WY          